Intermodal nx_graph can be loaded from OSM [example](graph_generator.ipynb)

In [1]:
from src.objectnat import get_adjacency_matrix
import networkx as nx
import geopandas as gpd
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns',15)

nx_graph = nx.read_graphml("examples_data/taraIntermodal.graphml")  # Data inside the file is in the local CRS EPSG:32643
buildings = gpd.read_file("examples_data/tara_buildings.geojson")  # Data inside the file is in the global CRS EPSG:4326
services = gpd.read_file("examples_data/tara_kinder.geojson")  # Data inside the file is in the global CRS EPSG:4326
matrix: pd.DataFrame = get_adjacency_matrix(buildings_from=buildings,
                              services_to=services,
                              nx_graph=nx_graph,
                              weight="time_min",  # weight  could be only "time_min" or "length_meter"
                              city_crs=32643)  # CRS is explicitly specified 
display(matrix)

2024-03-07 16:24:17.987 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:85 - Creating adjacency matrix based on provided graph...
2024-03-07 16:24:18.245 | DEBUG    | dongraphio.base_models:get_adjacency_matrix:92 - Calculating distances from buildings to services ...
2024-03-07 16:24:18.268 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:93 - Adjacency matrix done!


,0,1,2,3,4,5,6,...,6159,6160,6161,6162,6163,6164,6165
0,47.0,33.7,27.9,31.2,29.5,39.9,46.6,...,83.4,55.5,54.1,54.1,45.2,50.6,37.3
1,30.9,28.8,20.4,28.0,15.4,23.8,30.5,...,67.6,39.7,38.3,38.3,29.1,34.5,21.2
2,30.3,18.9,9.6,16.4,12.1,23.2,29.9,...,66.8,38.9,37.5,37.5,28.5,33.9,20.6
3,40.3,25.0,20.6,22.5,22.2,33.2,39.9,...,76.8,48.9,47.5,47.5,38.5,43.9,30.6
4,23.7,14.3,6.7,16.0,0.0,16.6,23.3,...,64.1,36.2,34.8,34.8,21.9,27.3,14.0
5,4.8,49.8,42.7,52.0,37.8,12.9,1.2,...,86.0,58.1,56.7,56.7,3.0,8.4,16.5
6,43.3,27.1,24.2,24.6,25.8,36.2,42.9,...,80.0,52.1,50.7,50.7,41.5,46.9,33.6
7,25.2,29.4,21.0,30.3,16.1,18.1,24.8,...,60.8,32.9,31.5,31.5,23.4,28.8,15.5
8,33.6,36.9,28.5,37.8,23.6,26.5,33.2,...,60.9,41.2,39.8,39.8,31.8,37.2,23.9


In [2]:
buildings.index = buildings["building_id"] # Set unique id attribute as DataFrame indexes, if you have one
services.index = services["service_id"]
matrix = get_adjacency_matrix(buildings_from=buildings,
                              services_to=services,
                              nx_graph=nx_graph,
                              weight="time_min",  # Weight  could be only "time_min" or "length_meter"
                              city_crs=32643)
display(matrix)

2024-03-07 16:24:18.295 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:85 - Creating adjacency matrix based on provided graph...
2024-03-07 16:24:18.511 | DEBUG    | dongraphio.base_models:get_adjacency_matrix:92 - Calculating distances from buildings to services ...
2024-03-07 16:24:18.525 | INFO     | dongraphio.dongraphio:get_adjacency_matrix:93 - Adjacency matrix done!


building_id,1721225,1721201,1721203,1721204,1721214,1721222,1721223,...,1730545,1730546,1730547,1730548,1721226,1721227,1721245
service_id,,,,,,,,,,,,,,,
1407512,47.0,33.7,27.9,31.2,29.5,39.9,46.6,...,83.4,55.5,54.1,54.1,45.2,50.6,37.3
1407507,30.9,28.8,20.4,28.0,15.4,23.8,30.5,...,67.6,39.7,38.3,38.3,29.1,34.5,21.2
1407513,30.3,18.9,9.6,16.4,12.1,23.2,29.9,...,66.8,38.9,37.5,37.5,28.5,33.9,20.6
1407505,40.3,25.0,20.6,22.5,22.2,33.2,39.9,...,76.8,48.9,47.5,47.5,38.5,43.9,30.6
1407511,23.7,14.3,6.7,16.0,0.0,16.6,23.3,...,64.1,36.2,34.8,34.8,21.9,27.3,14.0
1407508,4.8,49.8,42.7,52.0,37.8,12.9,1.2,...,86.0,58.1,56.7,56.7,3.0,8.4,16.5
1407510,43.3,27.1,24.2,24.6,25.8,36.2,42.9,...,80.0,52.1,50.7,50.7,41.5,46.9,33.6
1407506,25.2,29.4,21.0,30.3,16.1,18.1,24.8,...,60.8,32.9,31.5,31.5,23.4,28.8,15.5
1407509,33.6,36.9,28.5,37.8,23.6,26.5,33.2,...,60.9,41.2,39.8,39.8,31.8,37.2,23.9


In [4]:
matrix.to_csv("examples_data/adjacency_matrix.csv") #index=False to get rid of unnamed column